# DSIR SQL Technical Interview Practice
-----
Questions from an undisclosed company’s technical interview for “Business Intelligence Manager”, courtesy of a DSI alumn  
Solutions by: Kelly Slatery

In [1]:
import pandas as pd
from pandasql import sqldf

## Tables

In [2]:
EMPLOYEE = pd.DataFrame({
    'EMPLOYEE_ID': [601, 602, 603, 604, 605, 606],
    'FIRSTNAME': ['John', 'Bill', 'Mark', 'Lynda', 'Abby', 'Alex'],
    'LASTNAME': ['Smith', 'Wright', 'Jones', 'Rossi', 'Azuelo', 'Garcia'],
    'LOCATION' : ['Atlanta', 'Houston', 'Boston', 'Atlanta', 'Dallas', 'Dallas'],
    'TITLE' : ['Executive', 'Lead', 'Executive', 'Manager', 'Associate', 'Associate']    
})

In [3]:
COMPENSATION = pd.DataFrame({
    'EMPLOYEE_ID': [601, 602, 603, 604, 605, 606],
    'SALARY' : [50000, 30000, 45000, 40000, 25000, 25000],
    'BONUS' : [5000, 3000, 4000, 4000, 0, 0],
    'ANNUAL_PTO_DAYS' : [20, 14, 20, 16, 10, 10],
    'RETIRE_MATCHING' : ['Y', 'N', 'Y', 'Y', 'N', 'N']    
})

In [4]:
EMPLOYEE

,EMPLOYEE_ID,FIRSTNAME,LASTNAME,LOCATION,TITLE
0,601,John,Smith,Atlanta,Executive
1,602,Bill,Wright,Houston,Lead
2,603,Mark,Jones,Boston,Executive
3,604,Lynda,Rossi,Atlanta,Manager
4,605,Abby,Azuelo,Dallas,Associate
5,606,Alex,Garcia,Dallas,Associate


In [5]:
COMPENSATION

,EMPLOYEE_ID,SALARY,BONUS,ANNUAL_PTO_DAYS,RETIRE_MATCHING
0,601,50000,5000,20,Y
1,602,30000,3000,14,N
2,603,45000,4000,20,Y
3,604,40000,4000,16,Y
4,605,25000,0,10,N
5,606,25000,0,10,N


## Queries

#### 1. Using the employee and the compensation tables above – write a query that will provide a count of non-executive employees by the RETIRE_MATCHING column.

In [6]:
sql = """
SELECT c.RETIRE_MATCHING AS 'Retired', 
        COUNT(e.EMPLOYEE_ID) AS 'Count of Non-Executive Employees'
FROM EMPLOYEE AS e
INNER JOIN COMPENSATION AS c
    ON c.EMPLOYEE_ID = e.EMPLOYEE_ID
WHERE e.TITLE <> 'Executive'
GROUP BY c.RETIRE_MATCHING;
"""

df = sqldf(sql)
df.head()

,Retired,Count of Non-Executive Employees
0,N,3
1,Y,1


#### 2. Write a SQL Query To Fetch Unique Values Of TITLE From EMPLOYEE Table.  

In [7]:
sql = """
SELECT DISTINCT TITLE
FROM EMPLOYEE;
"""

df = sqldf(sql)
df.head()

,TITLE
0,Executive
1,Lead
2,Manager
3,Associate


#### 3. List the 4 types of joins and basic description of each.

Source: https://www.w3schools.com/sql/sql_join.asp
- (INNER) JOIN: Returns records that have matching values in both tables
- LEFT (OUTER) JOIN: Returns all records from the left table, and the matched records from the right table
- RIGHT (OUTER) JOIN: Returns all records from the right table, and the matched records from the left table
- FULL (OUTER) JOIN: Returns all records when there is a match in either left or right table


#### 4.  Using the query below answer the following questions about the result set:
```
SELECT E.LOCATION, SUM(C.SALARY) AS SALARY
FROM EMPLOYEE E
,COMPENSATION C
WHERE E.EMPLOYEE_ID = C.EMPLOYEE_ID
AND E.TITLE <> ‘EXECUTIVE’
GROUP BY E.LOCATION
```
 
- What is the resulting sum of Salary for Atlanta?
- How many total columns are returned?

**ERROR: We cannot SELECT FROM two tables at once using a WHERE statement. We would need to join the two tables.**  
If there were a join with correct syntax (like above), this would return:
- Sum of salaries for non-executives in Atlanta: $40,000
- 2 columns


#### 5. Write a SQL query to return the employee first and lastname with the same salary (without hard coding the salary amount).  

In [8]:
# Chris and Julio
sql = """
SELECT e.FIRSTNAME, e.LASTNAME
FROM EMPLOYEE AS e
INNER JOIN COMPENSATION AS c1
    ON e.EMPLOYEE_ID = c1.EMPLOYEE_ID
INNER JOIN COMPENSATION AS c2
    ON c1.SALARY = c2.SALARY
        AND c1.EMPLOYEE_ID <> c2.EMPLOYEE_ID;
"""

df = sqldf(sql)
df.head()

,FIRSTNAME,LASTNAME
0,Abby,Azuelo
1,Alex,Garcia


In [9]:
# Query that returns the matching salaries
sql = """
SELECT SALARY, COUNT(SALARY)
FROM COMPENSATION
GROUP BY SALARY
HAVING COUNT(SALARY) > 1;
"""

df = sqldf(sql)
df.head()

,SALARY,COUNT(SALARY)
0,25000,2


In [10]:
# Using the above query to create a table to join on
sql = """
SELECT e.FIRSTNAME, e.LASTNAME
FROM EMPLOYEE AS e
INNER JOIN COMPENSATION AS c
    ON e.EMPLOYEE_ID = c.EMPLOYEE_ID
INNER JOIN (
    SELECT SALARY, COUNT(SALARY)
    FROM COMPENSATION
    GROUP BY SALARY
    HAVING COUNT(SALARY) > 1
) AS df 
    ON c.SALARY = df.SALARY;
"""

df = sqldf(sql)
df.head()

,FIRSTNAME,LASTNAME
0,Abby,Azuelo
1,Alex,Garcia


#### 6. Write a SQL query to return employees firstname, title, and location - whose SALARY lies between (inclusive) 30000 and 45000.


In [11]:
sql = """
SELECT e.FIRSTNAME, e.TITLE, e.LOCATION
FROM EMPLOYEE AS E
INNER JOIN COMPENSATION AS C
    ON e.EMPLOYEE_ID = c.EMPLOYEE_ID
WHERE c.SALARY BETWEEN 30000 AND 45000;
"""

df = sqldf(sql)
df.head()

,FIRSTNAME,TITLE,LOCATION
0,Bill,Lead,Houston
1,Mark,Executive,Boston
2,Lynda,Manager,Atlanta
